## Imports

In [1]:
import pandas as pd
import tensorflow as tf

## Read Data from csv

In [6]:
dataX = pd.read_csv('../data/train.csv',delimiter='\t')
validX = pd.read_csv('../data/valid.csv',delimiter='\t')

In [14]:
dataY = pd.read_csv('../data/train_target.csv',header=None)
validY = pd.read_csv('../data/valid_target.csv',header=None)

In [7]:
dataX.head()

,cli_pl_header,cli_pl_body,cli_cont_len,srv_pl_header,srv_pl_body,srv_cont_len,aggregated_sessions,bytes,net_samples,tcp_frag,...,application_latency,cli_tx_time,load_time,server_latency,proxy,sp_healthscore,sp_req_duration,sp_is_lat,sp_error,throughput
0,667,0,0,336,143151,143151,1,155494,1,0,...,2.195,0.0,766.522,1.097,0,0,0,0,0,202.856539
1,667,0,0,336,143151,143151,1,310988,1,0,...,2.988,0.0,766.496,1.494,0,0,0,0,0,405.726840
2,667,0,0,336,143151,143151,1,310988,1,0,...,2.209,0.0,766.544,1.104,0,0,0,0,0,405.701434
3,592,0,0,238,1269,1269,1,2401,1,0,...,3.278,0.0,293.212,1.639,0,0,0,0,0,8.188614
4,592,0,0,238,1269,1269,1,4802,1,0,...,3.292,0.0,293.205,1.646,0,0,0,0,0,16.377620


In [23]:
dataX.shape[1]

49

In [9]:
validX.head()

,cli_pl_header,cli_pl_body,cli_cont_len,srv_pl_header,srv_pl_body,srv_cont_len,aggregated_sessions,bytes,net_samples,tcp_frag,...,application_latency,cli_tx_time,load_time,server_latency,proxy,sp_healthscore,sp_req_duration,sp_is_lat,sp_error,throughput
0,1179,0,0,425,4595,4595,1,13234,1,0,...,22.642,0.000,24.510,11.321,0,0,0,0,0,539.942880
1,1179,0,0,425,4595,4595,1,6609,1,0,...,22.647,0.000,24.425,12.330,0,0,0,0,0,270.583419
2,1104,0,0,548,4595,4595,1,6723,1,0,...,23.263,0.000,401.409,11.631,1,0,0,0,0,16.748503
3,1179,0,0,425,4595,4595,1,13234,1,0,...,21.733,0.000,24.023,10.866,0,0,0,0,0,550.888732
4,2055,0,0,326,7324,0,1,10867,1,0,...,73.733,0.009,74.765,6.044,0,0,0,0,0,145.348759


In [15]:
dataY.head()

,0
0,8
1,8
2,8
3,5
4,5


In [18]:
validY.describe()

,0
count,761179.000000
mean,2.271618
std,5.751112
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,42.000000


# Build an MLP
_______________________

### params

In [16]:
lr = 0.001
train_steps = 15
batch_size = 100
display_step = 1

### Network params

In [31]:
input_dim = dataX.shape[1]
num_classes = dataY.max()[0]
num_hidden_1 = 2*input_dim
num_hidden_2 = input_dim // 2

### Graph Input

In [32]:
x = tf.placeholder(tf.float32, shape= [None,input_dim], name='X')
y = tf.placeholder(tf.float32, shape= [None,num_classes], name = 'Y')

### Weights and Biases

In [33]:
weights = {
    'h1' : tf.Variable(tf.random_normal(shape=[input_dim, num_hidden_1])),
    'h2' : tf.Variable(tf.random_normal(shape=[num_hidden_1, num_hidden_2])),
    'out': tf.Variable(tf.random_normal(shape=[num_hidden_2, num_classes]))
}
biases = {
    'h1' : tf.Variable(tf.random_normal(shape=num_hidden_1)),
    'h2' : tf.Variable(tf.random_normal(shape=num_hidden_2)),
    'out': tf.Variable(tf.random_normal(shape=num_classes))
}

### Create model

In [34]:
def mlp_model(_X, weights, biases):
    layer_1 = tf.matmul(_X, weights['h1']) + biases['h1']
    layer_2 = tf.matmul(layer1, weights['h2']) + biases['h2']
    return tf.matmul(layer2, weights['out']) + biases['out']

## Training Operations

In [ ]:
pred = mlp_model(x,weights,biases)
# softmax loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))